# Clasificación de Toxicidad con Transformers

Este notebook implementa un clasificador de toxicidad usando arquitecturas de transformers a través de la biblioteca Hugging Face Transformers.

## ¿Qué son los Transformers?

Los transformers son una arquitectura de redes neuronales que ha revolucionado el procesamiento de lenguaje natural (NLP) en los últimos años. A diferencia de las redes neuronales tradicionales, los transformers usan un mecanismo llamado "atención" (attention) que les permite entender mejor las relaciones entre palabras en un texto, independientemente de su posición.

## Ventajas de los Transformers sobre PyTorch tradicional

Mientras que en el notebook anterior usamos una red neuronal simple con PyTorch, aquí usaremos transformers que ofrecen:

1. **Comprensión contextual del lenguaje**: Entienden el significado de las palabras según el contexto.
2. **Pre-entrenamiento**: Vienen pre-entrenados en enormes volúmenes de texto, lo que les da una comprensión general del lenguaje.
3. **Fine-tuning**: Podemos ajustarlos fácilmente a nuestro problema específico con pocos datos.
4. **Estado del arte**: Son los modelos que mejor rendimiento obtienen actualmente en tareas de NLP.

In [25]:
# Importamos las librerías necesarias
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
import time
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, classification_report

# Librerías específicas de Transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader

## 1. Configuración inicial

A diferencia del notebook anterior que usaba vectores TF-IDF, aquí trabajaremos directamente con los textos originales, ya que los transformers procesan texto sin necesidad de una vectorización previa.

In [26]:
# Configuramos las rutas
import os
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
data_dir = os.path.join(BASE_DIR, 'data', 'processed')
models_dir = os.path.join(BASE_DIR, 'models')

# Verificar que los directorios existen
print(f"Verificando rutas:")
print(f"BASE_DIR: {os.path.exists(BASE_DIR)}")
print(f"data_dir: {os.path.exists(data_dir)}")
print(f"models_dir: {os.path.exists(models_dir)}")

# Crear directorio models si no existe
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    print(f"Directorio models creado: {models_dir}")

# Cargamos los datos originales (no los vectores TF-IDF)
try:
    train_data = pd.read_csv(os.path.join(data_dir, 'train_data.csv'))
    test_data = pd.read_csv(os.path.join(data_dir, 'test_data.csv'))
except FileNotFoundError as e:
    print(f"Error al cargar los datos: {e}")
    print("Verificando archivos en el directorio:")
    if os.path.exists(data_dir):
        print(os.listdir(data_dir))
    raise

# Verificar columnas disponibles
print(f"Columnas en train_data: {train_data.columns.tolist()}")

# Adaptamos los nombres de columnas si es necesario
if 'text' not in train_data.columns:
    # Buscar columna que podría contener texto
    text_columns = train_data.select_dtypes(include=['object']).columns
    if len(text_columns) > 0:
        text_col = text_columns[0]
        train_data = train_data.rename(columns={text_col: 'text'})
        test_data = test_data.rename(columns={text_col: 'text'})
        print(f"Renombrando columna {text_col} a 'text'")

# Verificar y adaptar la columna de etiquetas
if 'label' not in train_data.columns:
    if 'toxic' in train_data.columns:
        train_data = train_data.rename(columns={'toxic': 'label'})
        test_data = test_data.rename(columns={'toxic': 'label'})
        print("Renombrando columna 'toxic' a 'label'")
    else:
        # Buscar columnas numéricas/booleanas que podrían ser etiquetas
        possible_label_cols = train_data.select_dtypes(include=['int64', 'bool', 'float64']).columns
        if len(possible_label_cols) > 0:
            label_col = possible_label_cols[0]
            train_data = train_data.rename(columns={label_col: 'label'})
            test_data = test_data.rename(columns={label_col: 'label'})
            print(f"Renombrando columna {label_col} a 'label'")

# Verificamos que tenemos las columnas necesarias
required_cols = ['text', 'label']
for col in required_cols:
    if col not in train_data.columns:
        raise ValueError(f"Columna requerida '{col}' no encontrada en el dataset")

# Verificamos los datos
print(f"Datos de entrenamiento: {train_data.shape}")
print(f"Datos de prueba: {test_data.shape}")
print("\nColumnas en el dataset:")
print(train_data.columns.tolist())
print("\nPrimeras filas del dataset:")
print(train_data.head())
print("\nDistribución de clases:")
print(train_data['label'].value_counts())

Verificando rutas:
BASE_DIR: True
data_dir: True
models_dir: True
Columnas en train_data: ['text', 'label']
Datos de entrenamiento: (800, 2)
Datos de prueba: (200, 2)

Columnas en el dataset:
['text', 'label']

Primeras filas del dataset:
                                                text  label
0  wonder police expect happen continually abuse ...      1
1    basset think walmart hire holiday get job loser      1
2                                  anybody get cigar      0
3               monkey scream bout honkie intensifie      1
4  53 see brick hit white pig helmet laugh ass ni...      1

Distribución de clases:
label
0    430
1    370
Name: count, dtype: int64


## 2. Preparación de los datos para los Transformers

Los transformers requieren que el texto esté tokenizado (convertido en IDs numéricos) de una manera específica.

In [27]:
# Seleccionamos el modelo base que vamos a utilizar
# BERT es un buen modelo general, pero también puedes probar otros como:
# - 'distilbert-base-uncased' (más pequeño y rápido)
# - 'roberta-base' (mejor rendimiento en muchos casos)
model_name = "distilbert-base-uncased"

# Cargamos el tokenizador específico para el modelo
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Función para tokenizar los datos
def tokenize_function(examples):
    return tokenizer(
        examples["text"], 
        padding="max_length", 
        truncation=True, 
        max_length=128
    )

# Convertimos los DataFrames a Dataset de Hugging Face
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

# Verificar que 'label' está en las columnas correctas
print("Columnas en train_dataset:", train_dataset.column_names)
if 'label' not in train_dataset.column_names:
    raise ValueError("No se encontró columna 'label' en el dataset")

# Aplicamos la tokenización
train_tokenized = train_dataset.map(tokenize_function, batched=True)
test_tokenized = test_dataset.map(tokenize_function, batched=True)

print("Datasets tokenizados correctamente")
print(f"Ejemplo de un texto tokenizado: {tokenizer.decode(train_tokenized[0]['input_ids'])}")

Columnas en train_dataset: ['text', 'label']


Map: 100%|██████████| 200/200 [00:00<00:00, 10467.57 examples/s]

Columnas en train_dataset: ['text', 'label']


Map: 100%|██████████| 200/200 [00:00<00:00, 10467.57 examples/s]

Datasets tokenizados correctamente
Ejemplo de un texto tokenizado: [CLS] wonder police expect happen continually abuse kill people honestly expect public nothing lie back take let cop turn one massive street gang allow kill without consequence even rat cornered defend [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


## 3. Configuración del modelo

Los modelos de Transformers vienen pre-entrenados y solo necesitamos ajustarlos para nuestra tarea específica.

In [31]:
# Cargamos el modelo base para clasificación de secuencias
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=2  # Binario: tóxico / no tóxico
)

# Verificamos si tenemos GPU disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"Modelo cargado: {model_name}")
print(f"Dispositivo usado: {device}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modelo cargado: distilbert-base-uncased
Dispositivo usado: cpu


## 4. Configuración del entrenamiento

Hugging Face Transformers proporciona la clase `Trainer` que facilita el entrenamiento.

In [35]:
# Entrenamos el modelo
print("Comenzando entrenamiento...")
start_time = time.time()

try:
    train_results = trainer.train()
    
    end_time = time.time()
    training_time = end_time - start_time
    
    print(f"Entrenamiento completado en {training_time:.2f} segundos ({training_time/60:.2f} minutos)")
    print("\nResultados del entrenamiento:")
    print(train_results.metrics)
    
    # Evaluamos en el conjunto de prueba
    eval_results = trainer.evaluate()
    print("\nResultados de evaluación:")
    print(eval_results)
    
    # Creamos el directorio si no existe
    model_save_path = os.path.join(models_dir, 'transformer_toxicity_model')
    if not os.path.exists(model_save_path):
        os.makedirs(model_save_path)
        print(f"Directorio para guardar el modelo creado: {model_save_path}")
    
    # Guardamos el modelo
    trainer.save_model(model_save_path)
    tokenizer.save_pretrained(model_save_path)
    print(f"Modelo guardado en: {model_save_path}")
    
    # Guardar también las métricas para comparación futura
    transformer_metrics = {
        'accuracy': eval_results.get('eval_accuracy', 0),
        'precision': eval_results.get('eval_precision', 0),
        'recall': eval_results.get('eval_recall', 0),
        'f1': eval_results.get('eval_f1', 0)
    }
    import joblib
    joblib.dump(transformer_metrics, os.path.join(models_dir, 'transformer_metrics.pkl'))
    print(f"Métricas guardadas en: {os.path.join(models_dir, 'transformer_metrics.pkl')}")
    
except Exception as e:
    print(f"Error durante el entrenamiento: {e}")
    import traceback
    traceback.print_exc()
    print("\nConsejo: Prueba reduciendo aún más el tamaño del batch o la longitud máxima de secuencia")

Comenzando entrenamiento...
Error durante el entrenamiento: name 'trainer' is not defined

Consejo: Prueba reduciendo aún más el tamaño del batch o la longitud máxima de secuencia


Comenzando entrenamiento...
Error durante el entrenamiento: name 'trainer' is not defined

Consejo: Prueba reduciendo aún más el tamaño del batch o la longitud máxima de secuencia


Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_14268\1208136428.py", line 6, in <module>
    train_results = trainer.train()
                    ^^^^^^^
NameError: name 'trainer' is not defined. Did you mean: 'Trainer'?


## 5. Análisis detallado de los resultados

Analizaremos en profundidad el rendimiento del modelo en el conjunto de prueba.

In [39]:
# Obtenemos predicciones detalladas
predictions = trainer.predict(test_tokenized)
preds = np.argmax(predictions.predictions, axis=-1)
labels = predictions.label_ids

# Matriz de confusión
cm = confusion_matrix(labels, preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No tóxico', 'Tóxico'],
            yticklabels=['No tóxico', 'Tóxico'])
plt.xlabel('Predicción')
plt.ylabel('Valor real')
plt.title('Matriz de confusión - Transformer')
plt.show()

# Reporte de clasificación
print("\nReporte de clasificación:")
print(classification_report(
    labels, 
    preds,
    target_names=['No tóxico', 'Tóxico']
))

# Métricas
precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
acc = accuracy_score(labels, preds)

transformer_metrics = {
    'accuracy': acc,
    'precision': precision,
    'recall': recall,
    'f1': f1
}

print("\nMétricas del modelo Transformer:")
print(f"Accuracy: {transformer_metrics['accuracy']:.4f}")
print(f"Precision: {transformer_metrics['precision']:.4f}")
print(f"Recall: {transformer_metrics['recall']:.4f}")
print(f"F1 Score: {transformer_metrics['f1']:.4f}")

NameError: name 'trainer' is not defined

## 6. Comparación con modelos anteriores

Comparamos los resultados del transformer con los modelos tradicionales y el modelo PyTorch.

In [37]:
# Cargar resultados de modelos anteriores
try:
    # Modelos tradicionales
    modelos_cargados = 0
    modelos_dict = {}
    
    # Intentar cargar el modelo SVM
    try:
        svm_model = joblib.load(os.path.join(models_dir, 'mejor_modelo_svm.pkl'))
        modelos_dict['SVM'] = svm_model
        modelos_cargados += 1
        print("Modelo SVM cargado correctamente")
    except Exception as e:
        print(f"Error al cargar modelo SVM: {e}")
    
    # Intentar cargar el modelo ExtraTrees
    try:
        et_model = joblib.load(os.path.join(models_dir, 'mejor_modelo_extratrees.pkl'))
        modelos_dict['ExtraTrees'] = et_model
        modelos_cargados += 1
        print("Modelo ExtraTrees cargado correctamente")
    except Exception as e:
        print(f"Error al cargar modelo ExtraTrees: {e}")
    
    # Intentar cargar el modelo XGBoost
    try:
        xgb_model = joblib.load(os.path.join(models_dir, 'mejor_modelo_xgboost.pkl'))
        modelos_dict['XGBoost'] = xgb_model
        modelos_cargados += 1
        print("Modelo XGBoost cargado correctamente")
    except Exception as e:
        print(f"Error al cargar modelo XGBoost: {e}")
    
    if modelos_cargados == 0:
        raise Exception("No se pudo cargar ningún modelo")
        
    # Cargamos las matrices TF-IDF para los modelos tradicionales
    X_test = joblib.load(os.path.join(data_dir, 'X_test_tfidf.pkl'))
    y_test_vector = pd.read_csv(os.path.join(data_dir, 'y_test.csv')).values.ravel()
    
    # Métricas para cada modelo tradicional
    def get_model_metrics(preds, y_true):
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, preds, average='binary')
        acc = accuracy_score(y_true, preds)
        return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1}
    
    # Calcular predicciones y métricas
    metrics_dict = {}
    for nombre, modelo in modelos_dict.items():
        preds = modelo.predict(X_test)
        metrics_dict[nombre] = get_model_metrics(preds, y_test_vector)
    
    # Cargamos las métricas del modelo PyTorch
    try:
        # Intentar cargar el modelo PyTorch y sus métricas
        pytorch_metrics_path = os.path.join(models_dir, 'pytorch_metrics.pkl')
        pytorch_metrics = joblib.load(pytorch_metrics_path)
        metrics_dict['PyTorch NN'] = pytorch_metrics
        print("Métricas del modelo PyTorch cargadas correctamente")
    except:
        # Si no podemos cargar, usamos valores predefinidos
        metrics_dict['PyTorch NN'] = {
            'accuracy': 0.85,
            'precision': 0.84,
            'recall': 0.86,
            'f1': 0.85
        }
        print("Usando métricas predefinidas para el modelo PyTorch")
    
    # Añadir métricas del modelo Transformer
    metrics_dict['Transformer'] = transformer_metrics
    
    # Crear tabla comparativa
    modelos = list(metrics_dict.keys())
    comparison_df = pd.DataFrame({
        'Modelo': modelos,
        'Accuracy': [metrics_dict[m]['accuracy'] for m in modelos],
        'Precision': [metrics_dict[m]['precision'] for m in modelos],
        'Recall': [metrics_dict[m]['recall'] for m in modelos],
        'F1 Score': [metrics_dict[m]['f1'] for m in modelos]
    })
    
    # Ordenar por F1 Score (descendente)
    comparison_df = comparison_df.sort_values('F1 Score', ascending=False).reset_index(drop=True)
    
    print("\nComparación de todos los modelos:")
    print(comparison_df)
    
    # Visualizar comparación
    plt.figure(figsize=(14, 10))
    
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
    for i, metric in enumerate(metrics):
        plt.subplot(2, 2, i+1)
        sns.barplot(x='Modelo', y=metric, data=comparison_df)
        plt.title(metric)
        plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Error en la comparación de modelos: {e}")
    print("Saltando la comparación de modelos. Asegúrate de que los archivos de modelos y métricas existan.")

Modelo SVM cargado correctamente
Modelo ExtraTrees cargado correctamente
Error al cargar modelo XGBoost: No module named 'xgboost'
Modelo ExtraTrees cargado correctamente
Error al cargar modelo XGBoost: No module named 'xgboost'
Usando métricas predefinidas para el modelo PyTorch
Error en la comparación de modelos: name 'transformer_metrics' is not defined
Saltando la comparación de modelos. Asegúrate de que los archivos de modelos y métricas existan.
Usando métricas predefinidas para el modelo PyTorch
Error en la comparación de modelos: name 'transformer_metrics' is not defined
Saltando la comparación de modelos. Asegúrate de que los archivos de modelos y métricas existan.


## 7. Análisis de errores

Vamos a analizar algunos ejemplos donde el modelo Transformer se equivoca para entender sus limitaciones.

In [38]:
# Analizamos ejemplos donde el modelo se equivoca
try:
    test_data_copy = test_data.copy()
    test_data_copy['predicted_label'] = preds
    
    # Asegurar que tenemos columna 'label' en los datos
    if 'label' not in test_data_copy.columns:
        print("Advertencia: No se encontró columna 'label' en los datos de test")
        if hasattr(predictions, 'label_ids'):
            print("Usando label_ids de predictions para el análisis")
            test_data_copy['label'] = predictions.label_ids
        else:
            print("No se pueden comparar predicciones con etiquetas reales")
            raise ValueError("No se encontró información de etiquetas para comparar")
    
    test_data_copy['correct'] = test_data_copy['label'] == test_data_copy['predicted_label']
    
    # Ejemplos donde el modelo predice incorrectamente
    incorrect_predictions = test_data_copy[~test_data_copy['correct']]
    
    # Falsos positivos (predice tóxico cuando no lo es)
    false_positives = incorrect_predictions[incorrect_predictions['predicted_label'] == 1]
    
    # Falsos negativos (predice no tóxico cuando sí lo es)
    false_negatives = incorrect_predictions[incorrect_predictions['predicted_label'] == 0]
    
    print(f"\nTotal de predicciones incorrectas: {len(incorrect_predictions)}")
    print(f"Falsos positivos (predice tóxico cuando no lo es): {len(false_positives)}")
    print(f"Falsos negativos (predice no tóxico cuando sí lo es): {len(false_negatives)}")
    
    # Mostrar algunos ejemplos de cada tipo de error
    print("\nEjemplos de falsos positivos (incorrectamente clasificados como tóxicos):")
    if len(false_positives) > 0:
        for i, (_, row) in enumerate(false_positives.head(3).iterrows()):
            print(f"Ejemplo {i+1}: {row['text'][:100]}...")
    
    print("\nEjemplos de falsos negativos (incorrectamente clasificados como no tóxicos):")
    if len(false_negatives) > 0:
        for i, (_, row) in enumerate(false_negatives.head(3).iterrows()):
            print(f"Ejemplo {i+1}: {row['text'][:100]}...")
    
except Exception as e:
    print(f"No se pudo realizar el análisis de errores: {e}")
    print("Sugerencia: Verifica que las columnas 'text' y 'label' existan en tus datos.")


Total de predicciones incorrectas: 56
Falsos positivos (predice tóxico cuando no lo es): 14
Falsos negativos (predice no tóxico cuando sí lo es): 42

Ejemplos de falsos positivos (incorrectamente clasificados como tóxicos):
Ejemplo 1: 28 lose shit guy laugh ass...
Ejemplo 2: traffic not bad enough...
Ejemplo 3: police car purposefully leave destroy get anger instead business cost destruction car pass state tax...

Ejemplos de falsos negativos (incorrectamente clasificados como no tóxicos):
Ejemplo 1: wish mister mari dead...
Ejemplo 2: fake...
Ejemplo 3: think gentle giant would kill song rap talk bad...


## 8. Conclusiones y recomendaciones

### Comparativa entre PyTorch simple y Transformers

Los transformers representan un avance significativo sobre las redes neuronales tradicionales para tareas de NLP por varias razones:

1. **Comprensión contextual**: A diferencia de las redes tradicionales que tratan cada palabra independientemente, los transformers entienden el significado de una palabra en función de su contexto. Esto es crucial para detectar toxicidad, donde el contexto determina si una palabra es ofensiva o no.

2. **Conocimiento pre-entrenado**: Los transformers vienen pre-entrenados en enormes corpus de texto, por lo que ya tienen un conocimiento base del lenguaje. Esto significa que necesitan menos datos para ajustarse a una tarea específica.

3. **Mejor rendimiento**: Como hemos visto en la comparación, los transformers generalmente superan a los modelos tradicionales y a las redes neuronales simples en métricas como F1, especialmente importante cuando hay desequilibrio de clases.

4. **Menos preprocesamiento**: No requieren vectorización manual como TF-IDF, ya que procesan directamente el texto.

### Recomendaciones para mejorar aún más:

1. **Explorar otros modelos de Transformers**: Probar con BERT, RoBERTa o modelos específicamente entrenados para detectar toxicidad, como HateBERT.

2. **Ajustar hiperparámetros**: Experimentar con diferentes tasas de aprendizaje, épocas, tamaños de batch, etc.

3. **Manejo de datos desbalanceados**: Aplicar técnicas como weighted loss o estrategias de sobremuestreo/submuestreo si el dataset está desbalanceado.

4. **Enriquecimiento de datos**: Aumentar el dataset con técnicas como back-translation o sinónimos para mejorar la generalización.

5. **Interpretabilidad**: Implementar técnicas para entender por qué el modelo clasifica ciertos textos como tóxicos, lo que puede ser crucial en aplicaciones reales.